<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
#!pip install imbalanced-learn -q
#!pip install swifter - q

#!pip install lightgbm
#!pip install xgboost

In [2]:
import pandas as pd
from tqdm import notebook
import re
from time import time 
import nltk
import numpy as np
from numpy.random import default_rng
import transformers
from sklearn.model_selection import train_test_split,cross_val_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score

from __future__ import unicode_literals, print_function
from spacy.lang.en import English

In [3]:
from imblearn.under_sampling import RandomUnderSampler

In [4]:
from sklearn.model_selection import TunedThresholdClassifierCV
from sklearn.model_selection import FixedThresholdClassifier

In [5]:
import torch
import transformers

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
from xgboost import XGBClassifier

In [8]:
import swifter

In [9]:
from sklearn.metrics import classification_report

In [10]:
from nltk.tokenize import sent_tokenize

In [11]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [12]:
#import spacy
#from nltk.corpus import stopwords
#nltk.download('stopwords')
#from spacy.pipeline import Lemmatizer
#!python -m spacy download en_core_web_sm
#nlp = spacy.load("en_core_web_sm")

In [13]:
#tokenizer = transformers.BertTokenizer(
#    vocab_file='datasets/ds_bert/vocab.txt')

In [133]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
#tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [15]:
#nlp = English()
#nlp.add_pipe('sentencizer')

In [134]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [17]:
tokenizer.encode('He ran. He fell.')

[101, 2002, 2743, 1012, 2002, 3062, 1012, 102]

In [18]:
tokenizer.added_tokens_encoder

{'[PAD]': 0, '[UNK]': 100, '[CLS]': 101, '[SEP]': 102, '[MASK]': 103}

In [19]:
for enc in tokenizer.encode('He ran. He fell. Aaaaaaaaaaa...'):
    display((enc,tokenizer.decode([f'{enc}'])))

(101, '[CLS]')

(2002, 'he')

(2743, 'ran')

(1012, '.')

(2002, 'he')

...

(11057, '##aa')

(1012, '.')

(1012, '.')

(1012, '.')

(102, '[SEP]')

In [20]:
RANDOM_STATE = 88

## Подготовка

In [21]:
start = time()
#df['text'].apply(lambda x: len(str(x).split()))
end = time()
end-start

0.0

In [22]:
url = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

In [23]:
df = pd.read_csv(url)

In [24]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [26]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [27]:
df['text'].duplicated().sum()

0

In [28]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [29]:
def clean_text(string):

    string = string.strip()
    string = re.sub(r'\[?https?://\S+\]?|\[?www\.\S+\]?', 'lnkt', string)
    string = re.sub(r'\[{1,2}.+\]{1,2}', 'brct', string)
    string = re.sub('\\d+\\.?', 'dgtl', string)
    string = re.sub(r'\n+(?=[a-zA-Z])', ' . ', string) #' '
    string = re.sub(r'\s+', ' ', string)
       
    return string.strip()

In [30]:
df['clean_text'] = df['text'].apply(lambda x:clean_text(x.lower()))

In [31]:
df['clean_text'].duplicated().sum()

510

In [32]:
df.drop_duplicates(subset='clean_text', inplace=True)

In [33]:
#df['clean_text'].duplicated().sum()

In [34]:
df['clean_text'].duplicated().sum()

0

In [35]:
df.reset_index(drop=True, inplace=True)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158782 entries, 0 to 158781
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   text        158782 non-null  object
 1   toxic       158782 non-null  int64 
 2   clean_text  158782 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.6+ MB


In [37]:
df['word_count'] = df['text'].apply(lambda x: x.count(' ') + 1)

In [38]:
df['sent_count'] = df['text'].apply(lambda x: len(sent_tokenize(x)))

In [39]:
df['len'] = df['text'].apply(lambda x: len(x))

In [40]:
#df[df['len']==df['len'].max()].head(10)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158782 entries, 0 to 158781
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   text        158782 non-null  object
 1   toxic       158782 non-null  int64 
 2   clean_text  158782 non-null  object
 3   word_count  158782 non-null  int64 
 4   sent_count  158782 non-null  int64 
 5   len         158782 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 7.3+ MB


In [42]:
#df['text_lower'] = df['mid_text'].str.lower()
#df.drop_duplicates(subset='text_lower', inplace=True)
#df.reset_index(drop=True, inplace=True)

In [43]:
def distribution_plt(df_list, name, val_list, plt_xlim=1):

    x_max = min([series.max() for series in df_list])* plt_xlim

    fig, ax = plt.subplots(2,1,figsize=(10,8))
    ax[0].boxplot([df for df in df_list], vert=False)

    ax[0].set_yticklabels(val_list)
    ax[0].set_xlabel(name)
    ax[0].set_xlim([0, x_max])

    ax[1].hist([df for df in df_list], histtype='stepfilled', bins=500)
    ax[1].set_ylabel('частота')
    ax[1].set_xlabel(name)
    ax[1].set_xlim([0, x_max])

    
    plt.suptitle(f'Ящики с усами и гистограмма для признака "{name}"')
    plt.legend(val_list);
    #plt.xlim((0,x_max))

In [44]:
name_dict = {'word_count' : 'количество слов',
 'sent_count' : 'количество предложений',
 'len' : 'количество символов'}

for col in ['word_count', 'sent_count', 'len']:
    
    distribution_plt(
        
        [df[df['toxic']==1][col], df[df['toxic']==0][col]],
        
        name_dict[col],
        
        ['токсичные\nкомментарии', 'не токсичные\nкомментарии'],
        
        0.1)

In [45]:
df[df['text'].str.contains('I ass. I ass.')]['text']

4703    do go fuck off bastard\nDo Yyou Have a life?\n...
Name: text, dtype: object

In [46]:
def logy(feats, target):
    lr = LogisticRegression(random_state=RANDOM_STATE,max_iter=1000,class_weight='balanced')
    lr.fit(feats, target)
    scores = cross_val_score(lr, feats, target, cv=5, scoring='f1_macro')
    return scores

In [47]:
df.shape

(158782, 6)

In [48]:
4096*16

65536

In [49]:
samp = df#.sample(n=6000, random_state=RANDOM_STATE)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(
    samp['text'],
    samp['toxic'],
    test_size = 0.16,
    random_state = RANDOM_STATE,
    stratify = samp['toxic'])

In [51]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((133376,), (25406,), (133376,), (25406,))

In [52]:
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train)) 

In [53]:
X_resample.shape, y_resample.shape

((27096, 1), (27096, 1))

In [54]:
res_df = X_resample.join(y_resample)

In [57]:
res_samp = res_df.sample(n=25000, random_state=RANDOM_STATE)

In [58]:
test_samp = pd.DataFrame(X_test).join(pd.DataFrame(y_test)).sample(n=25000, random_state=RANDOM_STATE)

In [59]:
res_samp.shape

(25000, 2)

In [60]:
test_samp.shape

(25000, 2)

In [61]:
X_resample.shape, y_resample.shape

((27096, 1), (27096, 1))

In [62]:
X_test.shape,y_test.shape

((25406,), (25406,))

In [220]:
#df['text'].loc[32281]

In [221]:
#print(str_cleaner(df['text'].loc[32281]))

In [224]:
#print(df['text'].loc[32281])

In [223]:
#tokenizer.decode(chunker(str_cleaner(df['text'].loc[32281])))

In [135]:
def chunker(string):

    string = clean_text(string) 
    
    sents_ = sent_tokenize(string)

    tokenized_sents = [tokenizer.encode(sent.strip(), add_special_tokens=True) for sent in sents_]

    usents=[]
    
    for sent in tokenized_sents:
        if sent not in usents and sent!=[101,1012,102]:
            usents.append(sent)
    
    if len(usents)>1:
       result =  sum(usents, [])
    elif len(usents)==1:
        result = usents[0]
    else:
        result = tokenized_sents[0]

    if len(set(result))/len(result)<0.1:
        result = list(dict.fromkeys(result[:-1]))#[:-1]
        
    
    
    return [101] + result[1:-1][:510] + [102]

def chunker(string):
    
    sents_ = sents(string)
    
    tokenz = tokenizer(sents_, add_special_tokens=True, padding=True, return_tensors='np')


    model = transformers.BertModel.from_pretrained('bert-large-uncased')#'distilbert-base-uncased')
    
    with torch.no_grad():
                    batch_embeddings = model(torch.LongTensor(tokenz['input_ids']), attention_mask=torch.LongTensor(tokenz['attention_mask']))
    #transformers.from_pretrained(distilbert-base-uncased)
    
    #batch_embeddings[0][:,0,:]
    
    return np.mean(batch_embeddings[0][:,0,:].numpy(),axis=0)

In [136]:
class BertTokenizer(object):

    def __init__(
        self,
        #tokenizer,
        #text=[],
        text,
        batch_size=1):
        
        self.text = text
        self.batch_size = batch_size
        self.model_class ,self.pretrained_weights = (
            transformers.BertModel, 'bert-base-uncased')
        #self.model_class ,self.pretrained_weights = (
        #    transformers.DistilBertModel, 'distilbert-base-uncased')

    
        
        self.model = self.model_class.from_pretrained(self.pretrained_weights)

    
    def get(self):

        
        tokenized = self.text.swifter.apply(lambda x: chunker(x))
        
        max_len = 0
        
        for tok in tokenized.values:
            if len(tok) > max_len:
                max_len = len(tok)

        padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
        attention_mask = np.where(padded != 0, 1, 0)
        
        batch_size = self.batch_size
        
        embeddings = []
        
        for i in notebook.tqdm(range(len(padded) // batch_size)):
                batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
                attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
                
                with torch.no_grad():
                    batch_embeddings = self.model(batch, attention_mask=attention_mask_batch)
                
                embeddings.append(batch_embeddings[0][:,0,:].numpy())
    
        features = np.concatenate(embeddings)

        return features

In [156]:
#df[df['text'].str.contains('I ass. I ass.')]['text']

In [155]:
#tokenizer.decode(chunker(df['text'][4703]))

In [154]:
#tokenizer.decode(chunker(df['text'][0]))

In [137]:
tokenizer.decode(chunker('someone needs to remove lott 71. 227. 167. 147'))

'[CLS] someone needs to remove lott dgtl dgtl dgtl dgtl [SEP]'

In [157]:
#BertTokenizer(df['text'][0:2]).get()

In [68]:
res_samp['text'].shape[0]

25000

In [69]:
X_train.shape[0]

133376

In [138]:
dembs_train = BertTokenizer(text=res_samp['text'], batch_size=50).get()

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [101]:
logy(dembs_train, res_samp['toxic'])

array([0.90499948, 0.89499859, 0.89465875, 0.89533315, 0.89899909])

In [139]:
np.array([0.90499948, 0.89499859, 0.89465875, 0.89533315, 0.89899909]).mean()

0.897797812

In [140]:
np.array([0.90749332, 0.8999999 , 0.89248815, 0.88999989, 0.89946921]).mean()

0.8978900940000001

In [141]:
#0.80789152, 0.80608067, 0.80544205, 0.79700029, 0.81400535

In [142]:
dembs_test = BertTokenizer(text=test_samp['text'], batch_size=50).get()

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [143]:
logy(dembs_train, res_samp['toxic'])

array([0.90259912, 0.89318754, 0.9041999 , 0.89579996, 0.89519973])

In [94]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train 
)

sample_weights

array([0.55718584, 0.55718584, 0.55718584, ..., 0.55718584, 4.87171159,
       0.55718584], shape=(30000,))

In [425]:
y_train.value_counts(normalize=1)

toxic
0    0.898439
1    0.101561
Name: proportion, dtype: float64

In [80]:
def f1_eval(y_true, y_pred,):
    #y_true = dtrain.get_label()
    #err = 1 - f1_score(grid_pred, y_test, average=None)[1]
    err = f1_score(y_pred, y_true, average=None)[1]
    return  err

In [81]:
def f1_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    err =1 - f1_score(y_true, y_pred, average=None)[1]

    return 'err', err

In [82]:
from sklearn.metrics import make_scorer
my_func = make_scorer(f1_eval, greater_is_better=True)

In [144]:
debs_no_dups = pd.DataFrame(dembs_train).join(pd.DataFrame(res_samp.reset_index()['toxic']))
#dembs_train, res_samp['toxic']

In [145]:
#debs_no_dups.head()

In [146]:
debs_no_dups.shape

(25000, 769)

In [147]:
debs_no_dups.duplicated().sum()

22

In [148]:
debs_no_dups.drop_duplicates(inplace=True)

In [ ]:
debs_no_dups.shape

In [149]:
pipe = Pipeline([('models',[LogisticRegression(random_state=RANDOM_STATE,class_weight='balanced')])])

In [150]:
param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, class_weight='balanced')],
        'models__solver': ['newton-cg', 'liblinear'],
        #'models__class_weight':,
        #'models__C': [0.1, 1]#,
        #'models__penalty': ['l1', 'l2']
          
    },

#    {
#        'models': [SVC(random_state=RANDOM_STATE, class_weight='balanced')],
#        'models__degree': range(3,8),
#        'models__kernel': ['poly','rbf','sigmoid']
#    },
    {
        'models': [XGBClassifier(
            random_state=RANDOM_STATE#,
            #scale_pos_weight = 0.897367
            
            )],
        'models__n_estimators':[500],
        'models__max_depth':[7],
        'models__learning_rate':[0.7,0.1],
        'models__eval_metric': [f1_xgb]
    }
]

param_grid = [

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, class_weight='balanced')],
        'models__solver': ['newton-cg', 'liblinear'],
        #'models__class_weight':,
        'models__C': [0.1]#,
        #'models__penalty': ['l1', 'l2']
          
    }
]

In [151]:
grid_search = GridSearchCV(
    pipe, 
    param_grid, 
    cv=5,
    scoring=my_func,
    #scoring='f1_macro',
    n_jobs=-1
)
#grid_search.fit(dembs_train, res_samp['toxic'])
grid_search.fit(debs_no_dups.drop('toxic',axis=1), debs_no_dups['toxic'])

,estimator,Pipeline(step..._state=88)])])
,param_grid,"[{'models': [LogisticRegre...ndom_state=88)], 'models__solver': ['newton-cg', 'liblinear']}, {'models': [XGBClassifier...ree=None, ...)], 'models__eval_metric': [<function f1_...0011DB27DA8E0>], 'models__learning_rate': [0.7, 0.1], 'models__max_depth': [7], ...}]"
,scoring,make_scorer(f...hod='predict')
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [152]:
pd.DataFrame(grid_search.cv_results_)[['rank_test_score', 'mean_test_score','params']].sort_values('rank_test_score')#.iloc[1]['params']#.head(16)

,rank_test_score,mean_test_score,params
1,1,0.898180,{'models': LogisticRegression(class_weight='ba...
0,2,0.898126,{'models': LogisticRegression(class_weight='ba...
3,3,0.892111,"{'models': XGBClassifier(base_score=None, boos..."
2,4,0.886001,"{'models': XGBClassifier(base_score=None, boos..."


In [87]:
#lr = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
#lr.fit(debs_no_dups.drop('toxic', axis=1), debs_no_dups['toxic'])

In [88]:
#preds_hands = lr.predict(dembs_test)

In [192]:
#preds_hands = xgbc.predict(dembs_test)

In [164]:
#grid_search.best_estimator_

In [153]:
grid_pred = grid_search.best_estimator_.predict(dembs_test)#[:X_train.shape[0]])

In [154]:
#probs = pd.DataFrame(grid_search.best_estimator_.predict_proba(dembs_test))

In [155]:
f1_score(grid_pred, test_samp['toxic'],average=None)

array([0.94239959, 0.65202084])

In [156]:
y_test.shape

(25406,)

In [157]:
dembs_test.shape

(25000, 768)

In [158]:
print(classification_report(grid_pred, test_samp['toxic']))

              precision    recall  f1-score   support

           0       0.90      0.99      0.94     20442
           1       0.91      0.51      0.65      4558

    accuracy                           0.90     25000
   macro avg       0.91      0.75      0.80     25000
weighted avg       0.90      0.90      0.89     25000



In [92]:
classifier_tuned = TunedThresholdClassifierCV(
    grid_search.best_estimator_, scoring=my_func,
).fit(dembs_train, res_samp['toxic'])

classifier_tuned = TunedThresholdClassifierCV(
    lr, scoring=my_func,
).fit(dembs_train, res_samp['toxic'])

In [93]:
ct_preds = classifier_tuned.predict(dembs_test)

In [94]:
f1_score(ct_preds, test_samp['toxic'])

0.6233766233766234

In [95]:
classifier_tuned.best_threshold_

0.4242438103240505

In [ ]:
#f1_score(ct_preds, y_test, average=None)

In [211]:
del xgbc

In [172]:
classifier_other_threshold = FixedThresholdClassifier(
    grid_search.best_estimator_, threshold=0.838, response_method="predict_proba"
).fit(dembs_train, res_samp['toxic'])
f1_score(test_samp['toxic'], classifier_other_threshold.predict(dembs_test))#,average=None)#82

0.7457367311745545

In [190]:
pd.DataFrame(dembs_train).duplicated().sum()

np.int64(16)

In [186]:
f1_score(y_test, classifier_other_threshold.predict(dembs_test))#82

0.7115547284723397

In [ ]:
cl_preds_adj = classifier_tuned.predict(dembs_val)

In [ ]:
f1_score(cl_preds_adj,y_val)

In [ ]:
res =pd.DataFrame({'a':grid_pred, 'b':test_sample['toxic']})

In [ ]:
pd.DataFrame(probs[0])

In [ ]:
y_test.shape

In [ ]:
pd.DataFrame({'proba':list(probs[1])},
              {'predicted':grid_pred})

In [ ]:
probs_res = pd.DataFrame(zip(y_test,probs[1],grid_pred),columns=['toxic', 'proba', 'pred'])

In [ ]:
probs_res[(probs_res['toxic']!=probs_res['pred']&(probs_res['toxic']==1))]['proba']

In [ ]:
X_test.iloc[1961]

In [ ]:
X_train.apply(lambda x: len(x)).hist()

In [ ]:
X_train.shape

In [ ]:
X_resample.shape

In [ ]:
X_train[X_train.apply(lambda x: len(x))>2000]

In [ ]:
res.shape

In [ ]:
df['text'].

In [ ]:
df[(df.index.isin(res[res['a']!=res['b']].index))&(df['toxic']==1)]['text'].apply(lambda x: tokenizer.decode(chunker(x)))#.sample()



#['b'].value_counts()

In [ ]:
tokenizer.decode(chunker(df.loc[128933]['text']))

In [ ]:
df.loc[128933]['text']

## Обучение

In [ ]:
#from phik.report import plot_correlation_matrix
#from phik import phik_matrix

In [ ]:
#df.info()

phik_overview = phik_matrix(df[['toxic','word_count']],verbose=False)

plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Blues',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(18, 14)
) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_lemm'], 
    df['toxic'], 
    random_state=RANDOM_STATE,
    stratify=df['toxic']
)

In [ ]:
sampler = RandomUnderSampler(random_state=RANDOM_STATE)
X_resample, y_resample = sampler.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train)) 


In [ ]:
y_resample.value_counts()

In [ ]:
pd.DataFrame(y_test).value_counts()

## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны